In [ ]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent= 4)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("factbench_dataset.csv")
bpdpdf = pd.read_csv("bpdp_dataset.csv")
factBenchDf = pd.read_csv("factbench_dataset.csv")
df = df.append(bpdpdf)
df = df.append(factBenchDf)
df.shape

In [ ]:
df['subject'] = df['subject'].apply(lambda x: x.split('/')[-1])
df['predicate'] = df['predicate'].apply(lambda x: x.split('/')[-1])
df['object'] = df['object'].apply(lambda x: x.split('/')[-1])
df

In [ ]:
df['subject'] = df['subject'].apply(lambda x: '<{}>'.format(x))
df['predicate'] = df['predicate'].apply(lambda x: '<{}>'.format(x))
df['object'] = df['object'].apply(lambda x: '<{}>'.format(x))
df.head(2)

In [ ]:
str_nodes_path = "/Users/palaniappanmuthuraman/Documents/GDRB/SourceCode-GFC/sample_data/gfc_str_nodes.tsv"
colnames=['subject','wordnet_description'] 
str_nodes_df = pd.read_csv(str_nodes_path,sep='\t',names = colnames)
str_nodes_df.head(1)
str_nodes_df.shape

In [ ]:
str_edges_path = "/Users/palaniappanmuthuraman/Documents/GDRB/SourceCode-GFC/sample_data/gfc_str_edges.tsv"
colnames=['subject','object','predicate']
str_edges_df = pd.read_csv(str_edges_path,sep='\t',names = colnames)
str_edges_df.head(1)

In [ ]:
str_ontology = "/Users/palaniappanmuthuraman/Documents/GDRB/SourceCode-GFC/sample_data/gfc_str_ontology.tsv"
colnames=['subject','object','predicate']
str_edges_df = pd.read_csv(str_edges_path,sep='\t',names = colnames)
str_edges_df.head(1)

In [ ]:
import io
with open("/Users/palaniappanmuthuraman/Documents/GDRB/SourceCode-GFC/sample_data/gfc_str_ontology.tsv","rb") as file:
    totalHashMap = {}
    fb = io.BufferedReader(file)
    while fb.readline():
        line = fb.readline().decode('utf-8')
        tokens = line.split("\t")
        child = tokens[0]
        hashMap = {'child':'','subchildMap':[]}
        hashMap['child'] = child
        for index,token in enumerate(tokens):
            if(token != child):
                hashMap['subchildMap'].append(token)
        totalHashMap[child] = hashMap
    print(len(totalHashMap))
    print(list(totalHashMap)[0])


In [ ]:
global objectFound 
objectFound = 0
global subejctFound 
subjectFound = 0

In [ ]:
def getSubjectObjectOntology(predicate,returnType):
    subject = ""
    object = ""
    if predicate == "<director>":
        subject =  "<wordnet_movie_106613686>"
        object =  "<wordnet_director_110014939>"
    elif predicate == "<starring>":
        subject = "<wordnet_movie_106613686>"
        object = "<wordnet_actor_109765278>"
    elif predicate == "<producer>":
        subject = "<wikicategory_American_film_producers>"
        object = "<wordnet_company_108058098>"
    elif predicate == "<productionCompany>":
        subject = "<wordnet_movie_106613686>"
        object = "<wordnet_company_108058098>"
    elif predicate == "<award>":
        subject = "<wordnet_person_100007846>"
        object = "<wordnet_award_106696483>"
    elif predicate == "<education>":
        subject = "<wordnet_person_100007846>"
        object = "<wordnet_university_108286569>"
    elif predicate == "<birthPlace>":
        subject = "<wordnet_person_100007846>"
        object = "<wordnet_city_108524735>"
    elif predicate == "<deathPlace>":
        subject = "<wordnet_person_100007846>"
        object = "<wordnet_city_108524735>"
    elif predicate == "<team>":
        subject = "<wordnet_player_110439851>"
        object = "<wordnet_team_108208560>"
    if(returnType == "subject"):
        return subject
    else:
        return object

In [ ]:
def getSubjectRelation(subject,predicate):
    data = str_nodes_df.loc[str_nodes_df['subject'] == subject]
    if(data.empty) : 
        return getSubjectObjectOntology(predicate=predicate,returnType="subject")
    else:
        global subjectFound
        subjectFound = subjectFound + 1
        return data['wordnet_description'].values[0]


In [ ]:
def getObjectRelation(object,predicate):
    objData = str_nodes_df.loc[str_nodes_df['subject'] == object]
    if(objData.empty):
        return getSubjectObjectOntology(predicate=predicate,returnType="object")
    else:
        global objectFound
        objectFound = objectFound + 1
        return objData['wordnet_description'].values[0]

In [ ]:
def getPredicate(predicate):
    if predicate == '<starring>':
        return "<actedIn>"
    elif predicate == '<director>':
        return "<directed>"
    elif predicate == '<producer>' | predicate == '<productionCompany>' | predicate == '<author>':
        return "<created>"
    elif predicate == "<award>":
        return "<hasWonPrize>"
    elif predicate == "<education>":
        return "<graduatedFrom>"
    elif predicate == "<birthPlace>":
        return "<wasBornIn>"
    elif predicate == "<deathPlace>":
        return "<diedIn>"
    elif predicate == "<team>":
        return "<isAffiliatedTo>"
    else:
        return predicate

In [ ]:
transformedDf = pd.DataFrame(columns = ['transformedData'])
for index,data in df.iterrows():
    transformedDf.loc[index] = data['subject'] + "," + getSubjectRelation(data['subject'],data['predicate']) + "," + getPredicate(data['predicate']) + "," + data['object'] + ","  + getObjectRelation(data['object'],data['predicate']) + "," + str(data['truth_value'])
with open("transformed_files.tsv",'w') as write_csv:
    write_csv.write(transformedDf.to_csv(sep='\t', index=False))
print(objectFound)
print(subjectFound)

In [ ]:
import csv
def checkFile():
    with open("transformed_files.tsv",'r') as f:
        reader = csv.reader(f, delimiter="\t")
        for i, line in enumerate(reader):
            if i == 0:
                continue
            tokens = line[0].split(',')
            if(len(tokens) != 6):
                print('This line {} have {} tokens'.format(i,len(tokens)))
                # raise Exception("Invalid Format")

In [ ]:
checkFile()